## [pyKNEEr](https://github.com/sbonaretti/pyKNEEr)

# [Segmenting MR Knee Images](https://sbonaretti.github.io/pyKNEEr/segmentation.html)

Segmentation is performed using an atlas-based algorithm. Given a segmented image (reference image), we use registration to segment each image (moving image) of the dataset. 

The segmentation consists of two parts:  
- Segment bone (used to initialize femoral cartilage segmentation)
- Segment cartilage

Each part is composed by 3 steps:  
- Step 1: *Register image to reference*. The moving image is registered to the reference image
- Step 2: *Invert transformation*. Transformations are inverted
- Step 3: *Warp reference mask to moving image*. Inverted transformations are applied to the mask of the reference image to obtain the mask of the moving image  

Registration, inversion, and warping are executed by calling *elastix*[1].

## Import packages

In [ ]:
%load_ext autoreload 
%autoreload 2

In [ ]:
from pykneer import pykneer_io          as io
from pykneer import segmentation_for_nb as segm

## Image information

Inputs:   
- `input_file_name` contains the list of the images to be segmented  
- `modality` can be "newsubject", "longitudinal", or "multimodal"
- `n_of_cores` is the number of cores used for computations (computations are parallelized for speed)

In [ ]:
input_file_name = "./image_list_segmentation_OAI1_DESS.txt"
modality        = "newsubject" # use "newsubject", "longitudinal", or "multimodal" 
n_of_cores      = 10

#### Read image data
- `image_data` is a dictionary (or struct), where each cell corresponds to an image. For each image, information such as paths and file names are stored  

In [ ]:
image_data = io.load_image_data_segmentation(modality, input_file_name)

## Reference image

The reference image is the number 9 of the current dataset. It was selected as a results of a convergence study described in the paper.
Here we check if femur mask and femur cartilage masks of the reference image are present

In [ ]:
segm.prepare_reference(image_data)

## Segment bone

#### 1. Register image to reference

In [ ]:
segm.register_bone_to_reference(image_data, n_of_cores)

#### 2. Invert transformations

In [ ]:
segm.invert_bone_transformations(image_data, n_of_cores)

#### 3. Warp reference mask to moving image

In [ ]:
segm.warp_bone_mask(image_data, n_of_cores)

## Segment cartilage

#### 1. Register image to reference

In [ ]:
segm.register_cartilage_to_reference(image_data, n_of_cores)

#### 2. Invert transformations

In [ ]:
segm.invert_cartilage_transformations(image_data, n_of_cores)

#### 3. Warp reference mask to moving image (optional)

In [ ]:
segm.warp_cartilage_mask(image_data, n_of_cores)

## Visualize segmentations
Visual check of the results. For each image (row), segmentation at medial, central, and lateral compartments

In [ ]:
segm.show_segmented_images(image_data)

### References

[1] Klein S., Staring M., Murphy K., Viergever M.A., Pluim J.P.W. [*elastix: A Toolbox for Intensity-Based Medical Image Registration*](http://elastix.isi.uu.nl/marius/downloads/2010_j_TMI.pdf) IEEE Transactions on Medical Imaging. vol. 29, no. 1, pp. 196 - 205, January. 2010. 

### Dependencies

In [ ]:
%load_ext watermark
print ("elastix 4.8.0\n")
%watermark -v -m -p SimpleITK,numpy,matplotlib,multiprocessing